# Type Hinting

In [ ]:
def get_full_name(first_name, last_name:str):
    full_name = first_name.title() + " " + last_name
    return full_name

print(get_full_name("john", "doe"))

Type hinting enables suggesttions. For example:

full_name = frist_name.sth

There will be a list of options

# FastAPI

In [ ]:
from enum import Enum

from fastapi import FastAPI


class ModelName(str, Enum):
    alexnet = "alexnet"
    resnet = "resnet"
    lenet = "lenet"


app = FastAPI()


@app.get("/models/{model_name}")
async def get_model(model_name: ModelName):
    if model_name is ModelName.alexnet:
        return {"model_name": model_name, "message": "Deep Learning FTW!"}

    if model_name.value == "lenet":
        return {"model_name": model_name, "message": "LeCNN all the images"}

    return {"model_name": model_name, "message": "Have some residuals"}

access /models/alexnet -> get "Deep Learning FTW!"

access /models/lenet -> get "LeCNN all the images"

access /models/resnet -> get "Have some residuals"

## Inheritance

In [1]:
class Animal:
    def speak(self):
        print("Some sound")

class Dog(Animal):
    pass

dog = Dog()
dog.speak()

Some sound


## Enum

In [ ]:
class ModelName(str, Enum):
    alexnet = "alexnet"
    resnet = "resnet"
    lenet = "lenet"

print(ModelName.alexnet)          # ModelName.alexnet
print(ModelName.alexnet.value)    # alexnet

## Parameters

If more than 1 layer, use

:path

In [ ]:
@app.get("/files/{file_path}")
async def read_file(file_path: str):
    return {"file_path": file_path}

/files/hello.txt                    -> Can access
/files/home/johndoe/myfile/txt      -> Cannot access

In [ ]:
@app.get("/files/{file_path:path}")
async def read_file(file_path: str):
    return {"file_path": file_path}

请求路径	                             匹配结果

/files/hello.txt	                    file_path = "hello.txt"

/files/home/johndoe/myfile.txt	        file_path = "home/johndoe/myfile.txt"

In [ ]:
@app.get("/items/{item_id}")
async def read_item(item_id: str, p: str, q: str | None = None):
    if q:
        return {"item_id": item_id, "q": q, "p": p}
    return {"item_id": item_id, "p": p}

item_id is a **path parameter**, since it's in {}: 

/items/apple

/items/123

/items/xyz

q is a **query parameter**:

/items/apple?q=hello&p=hola

Here, q is optional and p is not. Optional parameters must come after non-optional ones. 


## Request Body 请求体

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel # Verify input structure


class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None


app = FastAPI()


@app.post("/items/")
async def create_item(item: Item):
    return item

请求体的作用

1. 传递复杂数据结构

    JSON 可以表示嵌套对象、列表、字典，非常灵活。

    Pydantic 模型 Product 自动验证类型、必填字段、默认值。

2. 清晰的 API 语义

    路径参数 → 哪个资源

    请求体 → 更新后的内容

    查询参数 → 控制或可选参数

3. 安全性与扩展性

    可以加密或签名 JSON

    可以随时扩展字段而不破坏 API

## Extra Verification

### Query Parameters

In [ ]:
from typing import Annotated

from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/items/")
async def read_items(
    q: Annotated[
        str | None, Query(min_length=3, max_length=50, pattern="^fixedquery$")
    ] = None,
):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

The input can be a list. 

http://localhost:8000/items/?q=foo&q=bar

In [ ]:
from typing import Annotated

from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/items/")
async def read_items(q: Annotated[list[str] | None, Query()] = None):
    query_items = {"q": q}
    return query_items

If the query parameter is not a valid python variable, e.g. item-query

Use alias

In [ ]:
@app.get("/items/")
async def read_items(q: Annotated[str | None, Query(alias="item-query")] = None):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

### Path Parameters:

In [ ]:
from typing import Annotated

from fastapi import FastAPI, Path, Query

app = FastAPI()


@app.get("/items/{item_id}")
async def read_items(
    item_id: Annotated[int, Path(title="The ID of the item to get")],
    q: Annotated[str | None, Query(alias="item-query")] = None,
):
    results = {"item_id": item_id}
    if q:
        results.update({"q": q})
    return results

Unlike query parameters, path parameters are always non-optional

In [ ]:
def f(*, b, a): # After * key-word only
    ...
f(a=1, b=2)  # OK，顺序可调
f(1, 2)      # ❌ 错误

In [ ]:
async def read_items(
    *, q: str, item_id: int
):
    ...

Use * such that the order doesn't matter and all parameters must be passed with key-words

**Comparison**

gt: greater than

le: less than or equal

In [ ]:
@app.get("/items/{item_id}")
async def read_items(
    item_id: Annotated[int, Path(title="The ID of the item to get", ge=1)], q: str
):
    results = {"item_id": item_id}
    if q:
        results.update({"q": q})
    return results

### Pydantic Model

For lots of query parameters

GET /items/?limit=50&offset=10&order_by=updated_at&tags=python&tags=fastapi

    @app.get("/items/")
    async def read_items(limit: int = 100, offset: int = 0, order_by: str = "created_at"):
        return {"limit": limit, "offset": offset, "order_by": order_by}

Use Pydantic model to centralize restrictions

In [ ]:
class FilterParams(BaseModel):
    limit: int = Field(100, gt=0, le=100)  # 限制 limit 在 1~100 之间
    offset: int = Field(0, ge=0)           # offset >= 0
    order_by: Literal["created_at", "updated_at"] = "created_at" # Can only be these two
    tags: list[str] = []                   # 可以传多个 tags


In [ ]:
from typing import Annotated
from fastapi import Query

@app.get("/items/")
async def read_items(filter_query: Annotated[FilterParams, Query()]):
    return filter_query

    async def update_item(item_id: int, item: Item, importance: int):
        ...

PUT /items/42?importance=5

{
  "name": "Foo",
  "price": 42.0
}


But if want to have:

{
  "item": {...},
  "user": {...},

  "importance": 5

}

that is, if it's necessary to put **int** in json:

In [ ]:
from typing import Annotated

from fastapi import Body, FastAPI
from pydantic import BaseModel

app = FastAPI()


class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None


class User(BaseModel):
    username: str
    full_name: str | None = None


@app.put("/items/{item_id}")
async def update_item(
    item_id: int, item: Item, user: User, importance: Annotated[int, Body()]
):
    results = {"item_id": item_id, "item": item, "user": user, "importance": importance}
    return results

If want to have 

    {
    "item": {
        "name": "Foo",
        "description": "The pretender",
        "price": 42.0,
        "tax": 3.2
    }
}

Rather than:

    {
    "name": "Foo",
    "description": "The pretender",
    "price": 42.0,
    "tax": 3.2
}

In [ ]:
@app.put("/items/{item_id}")
async def update_item(item_id: int, item: Annotated[Item, Body(embed=True)]):
    results = {"item_id": item_id, "item": item}
    return results

## Field

Essentially the same with **Query, Path, Body**, which restricts input

In [ ]:
from pydantic import Field

app = FastAPI()


class Item(BaseModel):
    name: str
    description: str | None = Field(
        default=None, title="The description of the item", max_length=300
    )
    price: float = Field(gt=0, description="The price must be greater than zero")
    tax: float | None = None


@app.put("/items/{item_id}")
async def update_item(item_id: int, item: Annotated[Item, Body(embed=True)]):
    results = {"item_id": item_id, "item": item}
    return results

## Embeded Model

In [ ]:
class Image(BaseModel):
    url: str
    name: str


class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None
    tags: set[str] = set()
    image: Image | None = None


@app.put("/items/{item_id}")
async def update_item(item_id: int, item: Item):
    results = {"item_id": item_id, "item": item}
    return results

    {
    "name": "Foo",
    "description": "The pretender",
    "price": 42.0,
    "tax": 3.2,
    "tags": ["rock", "metal", "bar"],
    "image": {
        "url": "http://example.com/baz.jpg",
        "name": "The Foo live"
            }
    }

Check url

In [ ]:
from pydantic import BaseModel, HttpUrl

app = FastAPI()


class Image(BaseModel):
    url: HttpUrl
    name: str

## Cookie & Header

In [ ]:
from typing import Annotated

from fastapi import Cookie, FastAPI

app = FastAPI()


@app.get("/items/")
async def read_items(ads_id: Annotated[str | None, Cookie()] = None):
    return {"ads_id": ads_id}

In [ ]:
from typing import Annotated

from fastapi import FastAPI, Header

app = FastAPI()


@app.get("/items/")
async def read_items(user_agent: Annotated[str | None, Header()] = None):
    return {"User-Agent": user_agent}

## Respond Model

Mandate the returned model. Like def xxx: -> respond mondel

In [ ]:
from typing import Any

from fastapi import FastAPI
from pydantic import BaseModel, EmailStr

app = FastAPI()


class UserIn(BaseModel):
    username: str
    password: str
    email: EmailStr
    full_name: str | None = None


class UserOut(BaseModel):
    username: str
    email: EmailStr
    full_name: str | None = None


@app.post("/user/", response_model=UserOut)
async def create_user(user: UserIn) -> Any:
    return user

    def greet(name, age):
    print(name, age)

    data = {"name": "Alice", "age": 22}
    greet(**data)

    # Same as
    greet(name="Alice", age=22)

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel, EmailStr

app = FastAPI()


class UserIn(BaseModel):
    username: str
    password: str
    email: EmailStr
    full_name: str | None = None


class UserOut(BaseModel):
    username: str
    email: EmailStr
    full_name: str | None = None


class UserInDB(BaseModel):
    username: str
    hashed_password: str
    email: EmailStr
    full_name: str | None = None


def fake_password_hasher(raw_password: str):
    return "supersecret" + raw_password


def fake_save_user(user_in: UserIn):
    hashed_password = fake_password_hasher(user_in.password)
    user_in_db = UserInDB(**user_in.dict(), hashed_password=hashed_password)
    print("User saved! ..not really")
    return user_in_db


@app.post("/user/", response_model=UserOut)
async def create_user(user_in: UserIn):
    user_saved = fake_save_user(user_in)
    return user_saved

## Form

$ pip install python-multipart

Sometimes the client（客户端） request uses headers like: Content-Type: application/json

But some other time 

    POST /login/ HTTP/1.1
    Content-Type: application/x-www-form-urlencoded
In this case the body is:

    username=alice&password=123
So we use:

In [ ]:
from fastapi import Form

@app.post("/login_form/")
async def login_form(
    username: str = Form(..., description="User's login name", example="alice"), # Literally ...
    password: str = Form(..., min_length=6, max_length=32)
):
    return {"username": username}

In [ ]:
class FormData(BaseModel):
    username: str
    password: str


@app.post("/login/")
async def login(data: Annotated[FormData, Form()]):
    return data

## Upload File

In [ ]:
from typing import Annotated

from fastapi import FastAPI, File, UploadFile

app = FastAPI()


@app.post("/files/")
async def create_file(file: Annotated[bytes, File()]):
    return {"file_size": len(file)}


@app.post("/uploadfile/")
async def create_upload_file(file: UploadFile):
    return {"filename": file.filename}

In [6]:
from fastapi import FastAPI, File, UploadFile

app = FastAPI()

@app.post("/upload/")
async def upload_file(file: UploadFile = File(...)):
    contents = await file.read()
    return {"filename": file.filename, "size": len(contents)}


## Error Management

In [ ]:
from fastapi import FastAPI, HTTPException

app = FastAPI()

items = {"foo": "The Foo Wrestlers"}


@app.get("/items/{item_id}")
async def read_item(item_id: str):
    if item_id not in items:
        raise HTTPException(status_code=404, detail="Item not found")
    return {"item": items[item_id]}

## API doc visual elements

Group differnet endpoints

In [ ]:
class Item(BaseModel):
    name: str
    description: Union[str, None] = None
    price: float
    tax: Union[float, None] = None
    tags: Set[str] = set()


@app.post("/items/", response_model=Item, tags=["items"])
async def create_item(item: Item):
    return item


@app.get("/items/", tags=["items"])
async def read_items():
    return [{"name": "Foo", "price": 42}]


@app.get("/users/", tags=["users"])
async def read_users():
    return [{"username": "johndoe"}]

Markdown descriptions

In [ ]:
@app.post("/items/", response_model=Item, summary="Create an item")
async def create_item(item: Item):
    """
    Create an item with all the information:

    - **name**: each item must have a name
    - **description**: a long description
    - **price**: required
    - **tax**: if the item doesn't have tax, you can omit this
    - **tags**: a set of unique tag strings for this item
    """
    return item

Summary and description

In [ ]:
from typing import Set, Union

from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()


class Item(BaseModel):
    name: str
    description: Union[str, None] = None
    price: float
    tax: Union[float, None] = None
    tags: Set[str] = set()


@app.post(
    "/items/",
    response_model=Item,
    summary="Create an item",
    description="Create an item with all the information, name, description, price, tax and a set of unique tags",
)
async def create_item(item: Item):
    return item

## Jsonable Encoder

Some data types cannot be changed to json. Like datetime, date, Pydantic Model, set.

In [ ]:
from datetime import datetime
class Item(BaseModel):
    name: str
    timestamp: datetime

item = Item(name="Book", timestamp=datetime.now())

    print(item)
    name='Book' timestamp=datetime.datetime(2025, 10, 17, 15, 0, 0)
    print(item.dict())
    {'name': 'Book', 'timestamp': datetime.datetime(2025, 10, 17, 15, 0, 0)}


In [ ]:
from fastapi.encoders import jsonable_encoder

json_ready = jsonable_encoder(item)
print(json_ready)

    {'name': 'Book', 'timestamp': '2025-10-17T15:00:00'}


    @app.post("/store_result")
    def store_result(result: RAGResult):
        data = jsonable_encoder(result)
        db.insert(data)
        return {"message": "stored successfully"}